# Урок 3. Парсинг данных. HTML, Beautiful Soap
## Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    Наименование вакансии.
    Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

Ввод параметров запроса, а именно искомая должность и количество страниц для парсинга:

In [4]:
job_name = input('Введите должность: ') or 'Python developer'
pages = int(input('Введите количество страниц: ') or 1)

Введите должность: Инженер-конструктор
Введите количество страниц: 40


Проведем парсинг с hh.ru по искомой должности в цикле для каждой страницы (примечание - для 1 страницы параметр page не передаётся, иначе если страница всего одна - выдается пустая страница результата, возможно баг hh.ru, или какая то особенность непонятного мне назначения)

In [5]:
import requests, pprint
from bs4 import BeautifulSoup as bs
import pandas as pd
vacancy_result_info = []
headers = {"User-Agent" : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0'}
for page in range(1, pages + 1):
    params = {'text': job_name, 'only_with_salary':'true', 'salary' : '', 'ored_clusters' : 'true', 'enable_snippets' : 'true', 'customDomain':'1'}
    if page > 1:
        params['page']=str(page)
    response = requests.get('https://hh.ru/search/vacancy', headers=headers, params=params)
    soup = bs(response.content, 'html.parser')
    raw_vacancy_info = soup.find_all('div', attrs={'class':['vacancy-serp-item-body']})
    for vacancy in raw_vacancy_info:
        salary_text = vacancy.find('span', attrs={'data-qa':['vacancy-serp__vacancy-compensation']}).text.replace('\u202f', '')
        salary_text_list = salary_text.split()
        salary_unit = salary_text_list[-1]
        min_salary, max_salary = '',''
        for i in range(len(salary_text_list)):
            if salary_text_list[i] in ('от', 'from'):
                min_salary = salary_text_list[i+1]
            if salary_text_list[i] in ('до', 'to'):
                max_salary = salary_text_list[i+1]
            if salary_text_list[i] in ('–', '-'):
                min_salary = salary_text_list[i-1]
                max_salary = salary_text_list[i+1]
        vacancy_result_info.append([vacancy.find('a').text, min_salary, max_salary, salary_unit, vacancy.find('a')['href']])

Сформируем результат в DataFrame и сохраним как CSV:

In [6]:
vacancy_result = pd.DataFrame(vacancy_result_info, columns=['Должность', 'ЗП от', 'ЗП до', 'Валюта', 'Ссылка'])
vacancy_result.to_csv ('vacancy_result.csv', index= False )
vacancy_result

,Должность,ЗП от,ЗП до,Валюта,Ссылка
0,Инженер-конструктор,75000,95000,руб.,https://adsrv.hh.ru/click?b=367435&c=45&meta=v...
1,Ведущий Инженер-конструктор,110000,115000,руб.,https://adsrv.hh.ru/click?b=367440&c=5&meta=tz...
2,Инженер-конструктор,50000,120000,руб.,https://hh.ru/vacancy/77125033?from=vacancy_se...
3,Инженер - конструктор (Помощник инженера-конст...,60000,80000,руб.,https://hh.ru/vacancy/77017405?from=vacancy_se...
4,Инженер-конструктор металлоконструкций (Autode...,,600000,KZT,https://hh.ru/vacancy/76903100?from=vacancy_se...
...,...,...,...,...,...
775,Главный инженер-конструктор,110000,,руб.,https://hh.ru/vacancy/76230420?from=vacancy_se...
776,Инженер-конструктор,70000,,руб.,https://hh.ru/vacancy/76102276?from=vacancy_se...
777,Инженер-конструктор,60000,,руб.,https://hh.ru/vacancy/74241354?from=vacancy_se...
778,Ведущий инженер-конструктор КБ (укрытия),70000,,руб.,https://hh.ru/vacancy/76390223?from=vacancy_se...
